# Demo Mouse brain alignment and c-Fos analysis

Assumes:
- You have whole brain data from mesoSPIM (two channels, imaged with c-FOS and autofluorescence)
- You already ran BrainReg on them and have their registered atlas (See our config_brainreg parameters)
    - If not,launch CellSeg3D's napari GUI, import your autofluorescence image, Click on Plugins/Atlas Registration (brainreg), enter the parameters from the config (note that voxel size is specific to your experiment and specified in the metadata of your mesoSPIM experiment). Orientation as well. Can be checked using check orientation button. See brainreg doc: https://brainglobe.info/tutorials/tutorial-whole-brain-registration.html# for help.
- You should update the hardcoded paths yourself based on where the different files are located

Does:
1. Resizes the atlas based on the brain of the mouse
2. Match cfos brains with their registered homolog and cut out regions of interest
3. Remove autofluorescence
4. Remove borders after inference

Then:

- Thresholding and Remove artifacts and segmentation
- Count cells

In [9]:
config_bg = {
  "additional_images": [],
  "voxel_sizes": [5.26, 5.26, 5.0],
  "output_directory": output_directory_path,
  "atlas": "allen_mouse_25um",
  "n_free_cpus": 8,
  "hemisphere_selection": "full",
  "default_voxel_size_x": 5.26,
  "default_voxel_size_y": 5.26,
  "default_voxel_size_z": 5.0,
  "orientation": "sar",
  "preprocessing": "default",
  "sort_input_file": False,
  "save_orientation": False,
  "debug": False,
  "affine_n_steps": 6,
  "affine_use_n_steps": 5,
  "freeform_n_steps": 6,
  "freeform_use_n_steps": 5,
  "bending_energy_weight": 0.85,
  "grid_spacing": -10,
  "smoothing_sigma_reference": -1.0,
  "smoothing_sigma_floating": -1.0,
  "histogram_n_bins_floating": 128,
  "histogram_n_bins_reference": 128
}

# 0. Imports

In [1]:
import tifffile
import skimage
import time
import os
import glob
import sys
import pandas as pd
import numpy as np
import napari_cellseg3d
from napari_cellseg3d.dev_scripts import whole_brain_utils as wh

# 1. Resize atlas

In [2]:
main_exp_path = '/data/seb/CFOS_exp/TestBatch/'

In [20]:
#2023-10-30
#Sebastien B. Hausmann
#script order: 1

# Script to resize atlas based on related brain
from tifffile import imread, imwrite
from skimage.transform import resize
from bg_space import map_stack_to

# Prompt user for mouse name
mouse_name = input("Please enter the name of the mouse: ")

# Validate mouse name
if not mouse_name:
    raise ValueError("Mouse name cannot be empty.")

# Paths
brain_image_path_template = f'{main_exp_path}{mouse_name}*Mag1.25x*Ch488*.tiff'
brain_image_path = glob.glob(brain_image_path_template)

# Validate brain image path
if not len(brain_image_path):
    raise FileNotFoundError(f"Brain image for mouse {mouse_name} not found at path: {brain_image_path}")

corresponding_brain_atlas_path = f'{main_exp_path}reg_results_{mouse_name}/'

# Validate atlas path
if not os.path.exists(corresponding_brain_atlas_path):
    raise FileNotFoundError(f"Corresponding brain atlas directory for mouse {mouse_name} not found at path: {corresponding_brain_atlas_path}")

# Get actual shape of main image
try:
    brain_image_shape = imread(brain_image_path).shape
except Exception as e:
    raise IOError(f"Error reading brain image at {brain_image_path}. Error: {str(e)}")

# Get atlas
atlas_path = os.path.join(corresponding_brain_atlas_path, 'registered_atlas.tiff')
if not os.path.exists(atlas_path):
    raise FileNotFoundError(f"Atlas image not found at path: {atlas_path}")

try:
    atlas = imread(atlas_path)
except Exception as e:
    raise IOError(f"Error reading atlas image at {atlas_path}. Error: {str(e)}")

target_space = ["s", "a", "r"]
source_space = ["a", "s", "r"]

# Map atlas and resize it
atlas = map_stack_to(source_space, target_space, atlas, copy=False)
atlas = resize(atlas, brain_image_shape, order=0, preserve_range=True, anti_aliasing=False)

# Save image
output_path = os.path.join(corresponding_brain_atlas_path, f'registered_atlas_resize_{mouse_name}.tiff')
try:
    imwrite(output_path, atlas)
    print(f"Resized atlas saved successfully at: {output_path}")
except Exception as e:
    raise IOError(f"Error writing resized atlas to {output_path}. Error: {str(e)}")

Please enter the name of the mouse: ALUSKY
Resized atlas saved successfully at: /data/seb/CFOS_exp/TestBatch/reg_results_ALUSKY/registered_atlas_resize_ALUSKY.tiff


# 2. Match cfos brains with their registered homolog and cut out regions of interest

In [35]:
#2023-10-30
#Sebastien B. Hausmann
#script order: 2

# Script to match the cfos brains with their registered homolog brain and cut out regions of interest
# Saves it to a smaller file usable for cell segmentation
## IMPORTANT: Run this script twice, one time for autofluo and one time for cFOS only. You will be prompted when running it

# Add a variable to specify a specific mouse name (leave empty to process all files)
specific_mouse_name = input('Enter mouse name, can be left blank to do all: ') #can be leaved blank for all mice
cfos = input('cfos channel? (y/n): ')

cfos_images_path = main_exp_path
registered_brains_path = f'{main_exp_path}'
atlas_ID_dict = f'/{main_exp_path}allen_mouse_25um_csv_ref.csv'
cropped_regions_output_folder = f'/{main_exp_path}'

# Either enter Labels or keywords
regions_of_interest = input('Enter ONE allen brain atlas region of interest name (e.g., visual, retrosplenial, primary motor): ') #'retrosplenial' # 'visual' #'primary motor'

# Filter out your regions of interest to get the coresponding labels
df = pd.read_csv(atlas_ID_dict)
filtered_df = df[df['name'].str.contains(regions_of_interest, case=False, na=False)]
area_ids = np.array(filtered_df['id'])

# Find all tiff files in cfos_images_path containing a mouse name followed by Mag1.25x and containing Ch561.
if cfos=='y':
    pattern = os.path.join(cfos_images_path, '*Mag1.25x*Ch561*.tiff')
elif cfos=='n':
    pattern = os.path.join(cfos_images_path, '*Mag1.25x*Ch488*.tiff')
else:
    sys.exit()
    
cfos_tiff_files = glob.glob(pattern)

# Do the same for registered brains
pattern = os.path.join(registered_brains_path, 'registered*.tiff')
registered_brains_tiff_files = glob.glob(pattern)
   
for file_path in cfos_tiff_files:

    # Extract the mouse name from the file path
    mouse_name = os.path.basename(file_path).split('_Mag1.25x')[0]
    # Check if the specific_mouse_name is set and if the current file matches the mouse name
    if specific_mouse_name and specific_mouse_name != mouse_name:
        continue  # Skip this file as it doesn't match the specified mouse name

    start_time = time.time()

    print('brain:', file_path)
    # Load the tiff file
    brain_image = imread(file_path)

    # Find the corresponding atlas_image based on the mouse_name
    matched_atlas_path = [path for path in registered_brains_tiff_files if mouse_name in path]

    if matched_atlas_path and len(matched_atlas_path)==1:
        atlas_image = imread(matched_atlas_path[0]).data
    else:
        print(f"No registered brain found for mouse: {mouse_name}")
        break

    # Create a mask with regions of interest
    print('Create mask')
    mask = np.isin(atlas_image,area_ids) #

    # Corresponding indices
    print('Get indices')
    inds = np.where(mask)

    # 0 all the rest
    print('Zero brain image')
    brain_image[~mask] = 0

    # Finds mins and maxs to get the cropping coordinates
    mins = np.min(inds,axis=1)
    maxs = np.max(inds,axis=1)

    # Get the cropped region
    cropped_region = brain_image[mins[0]:maxs[0], mins[1]:maxs[1], mins[2]:maxs[2]]
    
    # Save the result to a desired location
    if cfos!='y':
        output_filename = f"{mouse_name}_ROI_{regions_of_interest}_autofluo.tiff"
    else:
        output_filename = f"{mouse_name}_ROI_{regions_of_interest}.tiff"
    output_path = os.path.join(cropped_regions_output_folder, output_filename)
    print('Saving now')
    imwrite(output_path, cropped_region)
    print('Done')
    print("--- %s seconds ---" % (time.time() - start_time))
    print('Next Brain...')

Enter mouse name, can be left blank to do all: ALUSKY
cfos channel? (y/n): n
Enter ONE allen brain atlas region of interest name (e.g., visual, retrosplenial, primary motor): primary motor
brain: /data/seb/CFOS_exp/TestBatch/ALUSKY_Mag1.25x_Tile0_Ch488_Sh0_Rot0.tiff
Create mask
Get indices
Zero brain image
Saving now
Done
--- 243.15659046173096 seconds ---
Next Brain...


# 3. Remove autofluorescence

In [36]:
#2023-10-30
#Sebastien B. Hausmann
#script order: 3

cropped_images_folder = main_exp_path

specific_mouse_name = input('Enter specific mouse name, or leave blank for all mice: ')
brain_region = input('Enter specific brain region: ')

# Find files in folder that contain autofluo in their name
pattern = os.path.join(cropped_images_folder, '*{}_autofluo.tiff'.format(brain_region))
autofluo_tiff_files = glob.glob(pattern)

for autofluo_file in autofluo_tiff_files:
    # Extract the mouse name from the file path
    mouse_name = os.path.basename(autofluo_file).split('_')[0]
    #print(mouse_name)
    # Check if the specific_mouse_name is set and if the current file matches the mouse name
    if specific_mouse_name and specific_mouse_name != mouse_name:
        continue  # Skip this file as it doesn't match the specified mouse name

    print('Remove background for:',autofluo_file)
    # Load images
    ## Load background image (autofluo)
    background_image = imread(autofluo_file)
    ## Load cfos image
    cfos_file = autofluo_file.replace('_autofluo', '')
    cfos_image = imread(cfos_file)

    # Convert both into int32 to allow correct sutraction
    background_image = background_image.astype(np.int32)
    cfos_image = cfos_image.astype(np.int32)

    # Background corrected image:
    corrected = cfos_image-background_image

    # Zero all the values lower than 0
    corrected[corrected<0] = 0
    
    corrected = corrected.astype(np.uint16)

    # Save the result to a desired location
    output_filename = autofluo_file.replace('_autofluo', '_backgroundCorrected')
    output_path = os.path.join(cropped_images_folder, output_filename)
    imwrite(output_path, corrected)  


Enter specific mouse name, or leave blank for all mice: ALUSKY
Enter specific brain region: primary motor
Remove background for: /data/seb/CFOS_exp/TestBatch/ALUSKY_ROI_primary motor_autofluo.tiff


# 3.5 Run inference using CellSeg3D plugin
- Open napari
- Run inference on file(s) cropped

# 4. Remove borders after inference

In [ ]:
#2023-10-30
#Sebastien B. Hausmann
#script order: 4 (after inference)

cropped_images_folder = main_exp_path
pred_image_folder = main_exp_path

# Find files
pattern = os.path.join(cropped_images_folder, '*.tiff')

tiff_files = glob.glob(pattern)

filtered_files = [file for file in tiff_files if not file.endswith("_autofluo.tiff") and not file.endswith("_backgroundCorrected.tiff")]

thickness_to_remove = 10

for _file in filtered_files:

    print('remove borders for:',_file)

    # Check if the file starts with "registered_atlas_resize_"
    if os.path.basename(_file).startswith('registered_atlas_resize_'):
        # Extract the mouse name (the part before the first underscore after "registered_atlas_resize_")
        mouse_name = os.path.basename(_file).split('_')[-1].split('.')[0]
        print('Mouse name extracted:', mouse_name)
        # Load images
        image = imread(_file)
    
    else:
        # Handle other cases if needed
        continue
    # Extract the mouse name from the file path
    pattern = os.path.join(pred_image_folder, '*_pred*.tif')
    pred_tiff_files = glob.glob(pattern)

    # Find the corresponding pred image based on the mouse_name
    pred_path = [path for path in pred_tiff_files if os.path.basename(_file).split('_')[-1].split('.')[0] in path]

    if pred_path and len(pred_path)==1:
        print('related pred file:',pred_path[0])
        pred_image = imread(pred_path[0]).data
    else:
        print(f"No registered brain found for mouse: {mouse_name}")
        break

    image = wh.extract_continuous_region(image)
    pred = wh.remove_boundaries_from_segmentation(pred_image, image_labels=image, thickness_num_iters=thickness_to_remove)

    # Save the result to a desired location
    output_filename = os.path.basename(pred_path[0]).replace('.tif', '_rmBorders.tif')#pred_path.replace('.tif', 'rmBorders.tif')
    output_path = os.path.join(pred_image_folder, output_filename)
    imwrite(output_path, pred)  


remove borders for: /data/seb/CFOS_exp/TestBatch/registered_atlas.tiff
remove borders for: /data/seb/CFOS_exp/TestBatch/registered_atlas_resize_ALUSKY.tiff
Mouse name extracted: ALUSKY
related pred file: /data/seb/CFOS_exp/TestBatch/ALUSKY_ROI_primary motor_SwinUNetR_pred_1_2023_11_22_18_04_17.tif


# Thresholding and Remove artifacts and segmentation

## Final part: Threshold all images in folder and count cells

In [10]:
from napari_cellseg3d.code_models.instance_segmentation import threshold, voronoi_otsu, binary_watershed, clear_small_objects, volume_stats, InstanceMethod
from napari_cellseg3d.utils import get_all_matching_files, resize
from tifffile import imread, imwrite
import napari
from napari.settings import get_settings
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
    # if ipy_interactive is false, each viewer will wait before continuing
# otherwise you'll immediately get 4 viewers.

settings = get_settings()
settings.application.ipy_interactive = False

In [14]:
removed_border_paths = Path.cwd()/'border_removed/'
result_path = Path.cwd()/'border_removed/'

In [15]:
def show_napari_viewer(image, result, labels=False):
    viewer = napari.view_image(result, colormap='inferno', opacity=0.7)
    if labels:
        viewer.add_labels(image)
    else:
        viewer.add_image(image, colormap='gray', opacity=0.7)
    viewer.dims.ndisplay = 3
    napari.run()

In [16]:
thresholded_images = []
image_paths = get_all_matching_files(removed_border_paths)
for file in image_paths:
    image = imread(file)
    result = threshold(image, 0.65)
    #show_napari_viewer(image,result)    
    thresholded_images.append(result)
    


In [17]:
for i, th_im in enumerate(thresholded_images):
    binaryzed_im = binary_watershed(th_im, thres_objects = 0.1, thres_seeding=0.1, thres_small=500, rem_seed_thres=5)
    result = np.where(binaryzed_im==0, th_im, 0)
    resized_result = resize(result, zoom_factors = (1/3,1,1))
    #show_napari_viewer(result,resized_result)
    segmented_image = voronoi_otsu(resized_result, spot_sigma=0.5, outline_sigma=0.5)
    #show_napari_viewer(segmented_image, resized_result, labels=True)

    stats = volume_stats(segmented_image)
    
    image_name = image_paths[i].stem
    df = pd.DataFrame(stats.get_dict())
    print('mouse_name: ', image_name.split('_')[0])
    print('region :' ,image_name.split('_')[2])
    print(df['Number objects'][0])
    print('saving here :', str(result_path)+'/'+image_name+'.csv')
    df.to_csv(str(result_path)+'/'+image_name+'.csv')
    
    # Save the segmented image as a TIFF file
    tiff_output_path = str(result_path) + '/' + image_name + '_segmented.tiff'
    print('Saving segmented image to:', tiff_output_path)
    imwrite(tiff_output_path, segmented_image.astype('float32'))  # Ensure correct data type

size: 209


0 invalid sphericities were set to NaN. This occurs for objects with a volume of 1 pixel.


mouse_name:  ZOKOR
region : primary motor
14210
saving here : /home/sebastien/Downloads/border_removed/ZOKOR_ROI_primary motor_SwinUNetR_pred_9_2023_11_22_18_40_15_rmBorders.csv
Saving segmented image to: /home/sebastien/Downloads/border_removed/ZOKOR_ROI_primary motor_SwinUNetR_pred_9_2023_11_22_18_40_15_rmBorders_segmented.tiff
